In [1]:
import open3d as o3d
import numpy as np
import os
from tqdm import tqdm
#import tensorflow as tf
import random

## Fix your path here

In [2]:
filepath_kri = ["/Users/kristofferkjaer/Desktop/mesh", "/Users/kristofferkjaer/Desktop/SPRING_MALE"]

filepath_jacob = ["/Users/J.S.Kirkeby_PC/OneDrive/Uni/4. Semester/Fagprojekt/Data/Female", "/Users/J.S.Kirkeby_PC/OneDrive/Uni/4. Semester/Fagprojekt/Data/Female"]

filepath_karo = ["/Users/KAROLINE/Desktop/Fagprojekt/Data/Female","/Users/KAROLINE/Desktop/Fagprojekt/Data/Male"]

In [3]:
data_paths = filepath_karo

N = 5

pcds_females = []
pcds_males = []
pcds = [pcds_females, pcds_males]

for data in range(len(data_paths)):
    files = []
    folder_to_view = data_paths[data]
    list = pcds[data]
    for file in os.listdir(folder_to_view):
        files.append(file)
    for i in tqdm(range(N)):
        file = (folder_to_view + "/" + str(files[i]))
        mesh = o3d.io.read_triangle_mesh(file)
        pcd = mesh.sample_points_poisson_disk(2048)
        #pcd = pcd.voxel_down_sample(voxel_size=0.5)
        list.append(pcd)

100%|██████████| 5/5 [00:03<00:00,  1.66it/s]


### Normals

In [4]:
normals = [[],[]]
for gender in range(len(pcds)):
    for i in range(len(pcds_females)):
        normals[gender].append(np.asarray(pcds[gender][i].normals))

np.shape(normals[0][0])

(2048, 3)

### Augmentation

In [5]:
def augment(points):
    points += tf.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float64)
    points = tf.random.shuffle(points)
    return points

### Rotation

### Convert to numpy array

In [6]:
pcds_females_arr = []
pcds_males_arr = []
pcds_arr = [pcds_females_arr, pcds_males_arr]


for points in range(len(pcds)): 
    for i in tqdm(range(len(pcds_females))):
        pcds_arr[points].append(np.asarray(pcds[points][i].points) )



100%|██████████| 5/5 [00:00<00:00, 59409.41it/s]


In [7]:
furthest_distance = 0

for points in range(len(pcds_arr)): 
    for i in range(len(pcds_males_arr)):
        distance = np.max(np.sqrt(np.sum(abs(pcds_arr[points][i])**2,axis=1)))
        if distance > furthest_distance:
            furthest_distance = distance
            
furthest_distance

1.0337664626283618

### Normalize (Update to normalize in regards to biggest person)

In [8]:
def normalize(points): # The input of this function is a point cloud object
    # This function normalizes a point cloud within a unit sphere
    points = np.asarray(points.points) 
    centroid = np.mean(points, axis=0)
    points -= centroid
    points / furthest_distance
    return points # The function return a numpy array of dimension nx3 where the points are normalized within a unit sphere

pcds_females_arr_norm = []
pcds_males_arr_norm = []
pcds_arr_norm = [pcds_females_arr_norm, pcds_males_arr_norm]


for points in range(len(pcds_arr)):
    for i in tqdm(range(len(pcds_females))):
        pcds_arr_norm[points].append(normalize(pcds[points][i]))

100%|██████████| 5/5 [00:00<00:00, 5197.40it/s]


In [9]:
furthest_distance = 0

for points in range(len(pcds_arr_norm)): 
    for i in range(len(pcds_males_arr)):
        distance = np.max(np.sqrt(np.sum(abs(pcds_arr_norm[points][i])**2,axis=1)))
        if distance > furthest_distance:
            furthest_distance = distance
            
furthest_distance

0.9879612861284487

# Centraliser

# Augmentation

### Rotation

In [10]:
test_person = pcds_females[2]

In [11]:
# Define the maximum angle in degrees for the random rotation
max_angle = 45

# Generate a random rotation axis and angle
axis = np.random.uniform(-1, 1, 3)
axis /= np.linalg.norm(axis)
angle = np.random.uniform(-max_angle, max_angle) * np.pi / 180  # convert degrees to radians

# Create the rotation matrix using Open3D's utility function
#rot_matrix = o3d.geometry.get_rotation_matrix_from_axis_angle(axis, angle)
R = mesh.get_rotation_matrix_from_xyz((np.pi / 2, np.pi / 2, np.pi / 2))


# Apply the rotation to the point cloud using Open3D's built-in transform function
test_rotate = test_person.rotate(R)

### To visualise point cloud

In [12]:
#o3d.visualization.draw_geometries([test_person,pcds_males[2]])